## import packages

In [1]:
import requests
import pandas as pd
import time

In [ ]:
api_key = 'replace api key here'

## 1. text search api

In [124]:
def call_text_search_api(fieldmask,
                         textquery,
                         api_key,
                         page_size=20,        # text search returns a max of 20 results per page
                         max_pages=3):        # text search returns a max of 60 results in total

    url = 'https://places.googleapis.com/v1/places:searchText'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    all_places = []
    next_page_token = None
    total_results = 0

    for page in range(max_pages):
        if next_page_token:
            params = {'nextPageToken': next_page_token}
            response = requests.get(url, headers=headers, params=params)
        else:
            data = {
                'textQuery': textquery,
                'pageSize': page_size
            }
            response = requests.post(url, headers=headers, json=data)   # post accepts a json request body which allows for more parameters to be specified
            
        result = response.json()

        num_results_page = len(result.get('places', []))
        total_results += num_results_page
        print(f"page {page + 1}: returned {num_results_page} results (total: {total_results})")

        if 'places' in result:
            all_places.extend(result['places'])

        next_page_token = result.get('nextPageToken')
        if not next_page_token:
            break

        time.sleep(3)

    df = pd.json_normalize(all_places) if all_places else pd.DataFrame()

    return df

### 1.1. searching for hawker stalls in newton food centre

In [125]:
fieldmask_1_1a = '*'   # retrieves all available fields
textquery_1_1a = 'hawker stalls in Newton Food Centre Singapore'

test_1_1a = call_text_search_api(fieldmask_1_1a, textquery_1_1a, api_key)

page 1: returned 11 results (total: 11)


In [126]:
test_1_1a

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,servesBeer,servesWine,outdoorSeating,websiteUri,allowsDogs,accessibilityOptions.wheelchairAccessibleSeating,regularOpeningHours.nextOpenTime,currentOpeningHours.nextOpenTime,servesCoffee,servesBreakfast
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJx3TNyV8Z2jERAFrgrjj3HVA,ChIJx3TNyV8Z2jERAFrgrjj3HVA,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-31 Food Centre, Sing...","[{'longText': '#01-31', 'shortText': '#01-31',...",4.4,https://maps.google.com/?cid=57730421201830118...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,places/ChIJdadh-esZ2jERLMyRFjaOLsI,ChIJdadh-esZ2jERLMyRFjaOLsI,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-51 Newton Food Centr...","[{'longText': '01-51', 'shortText': '01-51', '...",2.2,https://maps.google.com/?cid=13992277455244741...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,places/ChIJJczvxGIZ2jER4wtULz2lCbg,ChIJJczvxGIZ2jER4wtULz2lCbg,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-13 Newton Food Centr...","[{'longText': '01-13', 'shortText': '01-13', '...",3.7,https://maps.google.com/?cid=13261312259974761...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,places/ChIJu3wK9-sZ2jERz3jrVfP4sWM,ChIJu3wK9-sZ2jERz3jrVfP4sWM,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-27 Newton Food Centr...","[{'longText': '#01-27', 'shortText': '#01-27',...",3.6,https://maps.google.com/?cid=71837966046348883...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,places/ChIJu0Rc9-sZ2jERF49RylTbSEs,ChIJu0Rc9-sZ2jERF49RylTbSEs,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-25 Newton Food Centr...","[{'longText': '01-25', 'shortText': '01-25', '...",3.6,https://maps.google.com/?cid=54248269083858409...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,https://www.instagram.com/newton_old_signboard...,False,True,NaN,NaN,NaN,NaN
6,places/ChIJxcbG7-sZ2jERiyhLwTAA5i8,ChIJxcbG7-sZ2jERiyhLwTAA5i8,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-19 Newton Food Centr...","[{'longText': '01-19', 'shortText': '01-19', '...",4.7,https://maps.google.com/?cid=34514463738274223...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-06T08:30:00Z,2025-11-06T08:30:00Z,NaN,NaN
7,places/ChIJHaje8OsZ2jERp5RIkq3p2iU,ChIJHaje8OsZ2jERp5RIkq3p2iU,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-05 Newton Food Centr...","[{'longText': '01-05', 'shortText': '01-05', '...",3.0,https://maps.google.com/?cid=27277494560191008...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,places/ChIJ5SNx8OsZ2jERzcZTgn4OBSs,ChIJ5SNx8OsZ2jERzcZTgn4OBSs,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-11, Singapore 229495","[{'longText': '01-11', 'shortText': '01-11', '...",4.3,https://maps.google.com/?cid=30998998550296511...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,True,https://m.facebook.com/HaiYanBBQSeafood,NaN,NaN,2025-11-06T08:00:00Z,2025-11-06T08:00:00Z,False,NaN
9,places/ChIJU_PRvacZ2jERC

In [127]:
print(test_1_1a.columns)   # all available fields returned

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [128]:
test_1_1a[['displayName.text',   # select columns of interest
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
1,31 Heng Heng BBQ 兴兴烧烤,ChIJx3TNyV8Z2jERAFrgrjj3HVA,restaurant,"500 Clemenceau Ave N, #01-31 Food Centre, Sing...",1.311950,103.839577,4.4,[{'name': 'places/ChIJx3TNyV8Z2jERAFrgrjj3HVA/...,111,"[Monday: 12:00 – 10:30 PM, Tuesday: Closed, We...",NaN,NaN,NaN
2,Newton 51.,ChIJdadh-esZ2jERLMyRFjaOLsI,restaurant,"500 Clemenceau Ave N, #01-51 Newton Food Centr...",1.312100,103.839363,2.2,[{'name': 'places/ChIJdadh-esZ2jERLMyRFjaOLsI/...,62,"[Monday: 11:00 AM – 11:00 PM, Tuesday: 11:00 A...",1,10,NaN
3,Kwee Heng,ChIJJczvxGIZ2jER4wtULz2lCbg,restaurant,"500 Clemenceau Ave N, #01-13 Newton Food Centr...",1.311854,103.839706,3.7,[{'name': 'places/ChIJJczvxGIZ2jER4wtULz2lCbg/...,93,"[Monday: 9:00 AM – 9:00 PM, Tuesday: 9:00 AM –...",1,10,NaN
4,Alliance Seafood,ChIJu3wK9-sZ2jERz3jrVfP4sWM,restaurant,"500 Clemenceau Ave N, #01-27 Newton Food Centr...",1.311986,103.839729,3.6,[{'name': 'places/ChIJu3wK9-sZ2jERz3jrVfP4sWM/...,188,"[Monday: Closed, Tuesday: 11:30 AM – 10:30 PM,...",NaN,NaN,NaN
5,Newton Old Signboard 25,ChIJu0Rc9-sZ2jERF49RylTbSEs,restaurant,"500 Clemenceau Ave N, #01-25 Newton Food Centr...",1.312115,103.839389,3.6,[{'name': 'places/ChIJu0Rc9-sZ2jERF49RylTbSEs/...,68,"[Monday: Closed, Tuesday: 10:45 AM – 10:30 PM,...",1,10,NaN
6,Stingray Goh Chew Kee Seafood,ChIJxcbG7-sZ2jERiyhLwTAA5i8,restaurant,"500 Clemenceau Ave N, #01-19 Newton Food Centr...",1.311868,103.839835,4.7,[{'name': 'places/ChIJxcbG7-sZ2jERiyhLwTAA5i8/...,41,"[Monday: 4:30 – 11:30 PM, Tuesday: 4:30 – 11:3...",10,20,NaN
7,88 San Ren Cold & Hot Dessert,ChIJHaje8OsZ2jERp5RIkq3p2iU,restaurant,"500 Clemenceau Ave N, #01-05 Newton Food Centr...",1.311842,103.839394,3.0,[{'name': 'places/ChIJHaje8OsZ2jERp5RIkq3p2iU/...,37,"[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",1,10,NaN
8,Hai Yan BBQ Seafood,ChIJ5SNx8OsZ2jERzcZTgn4OBSs,restaurant,"500 Clemenceau Ave N, #01-11, Singapore 229495",1.311851,103.839553,4.3,[{'name': 'places/ChIJ5SNx8OsZ2jERzcZTgn4OBSs/...,62,"[Monday: Closed, Tuesday: 4:00 – 11:00 PM, Wed...",10,20,NaN
9,Hup Kee Fried Oyster Omelette,ChIJU_PRvacZ2jERC2QJ764xZRk,restaurant,"500 Clemenceau Ave N, Newton Circus, #01-73 Fo...",1.312241,103.839583,4.0,[{'name': 'places/ChIJU_PRvacZ2jERC2QJ764xZRk/...,111,"[Monday: Closed, Tuesday: Closed, Wednesday: 5...",1,10,NaN


In [129]:
fieldmask_1_1b = '*'
textquery_1_1b = 'hawker stalls in newton food centre singapore'   # text search is case sensitive

test_1_1b = call_text_search_api(fieldmask_1_1b, textquery_1_1b, api_key)

page 1: returned 1 results (total: 1)


In [16]:
test_1_1b

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [17]:
print(test_1_1b.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [24]:
test_1_1b[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


In [38]:
fieldmask_1_1c = '*'
textquery_1_1c = 'stalls in Newton Food Centre Singapore'

test_1_1c = call_text_search_api(fieldmask_1_1c, textquery_1_1c, api_key)

page 1: returned 1 results (total: 1)


In [39]:
test_1_1c

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [40]:
print(test_1_1c.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [41]:
test_1_1c[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


In [130]:
fieldmask_1_1d = '*'
textquery_1_1d = 'Newton Food Centre Singapore'

test_1_1d = call_text_search_api(fieldmask_1_1d, textquery_1_1d, api_key)

page 1: returned 1 results (total: 1)


In [131]:
test_1_1d

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [132]:
print(test_1_1d.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [133]:
test_1_1d[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


In [134]:
fieldmask_1_1e = '*'
textquery_1_1e = 'newton food centre'

test_1_1e = call_text_search_api(fieldmask_1_1e, textquery_1_1e, api_key)

page 1: returned 1 results (total: 1)


In [135]:
test_1_1e

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [136]:
print(test_1_1e.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [137]:
test_1_1e[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...


### 1.2. searching for a specific hawker stall in newton food centre

In [165]:
fieldmask_1_2 = '*'
textquery_1_2 = 'kwang kee teochew fish porridge in newton food centre'

test_1_2 = call_text_search_api(fieldmask_1_2, textquery_1_2, api_key)

page 1: returned 1 results (total: 1)


In [166]:
test_1_2

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,utcOffsetMinutes,adrFormatAddress,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY,ChIJPbO-7-sZ2jER1zjzFT3nUyY,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-20, Singapore 229495","[{'longText': '#01-20', 'shortText': '#01-20',...",4,https://maps.google.com/?cid=27618052460555286...,https://food.grab.com/sg/en/restaurant/kwang-k...,480,"<span class=""street-address"">500 Clemenceau Av...",...,https://www.google.com/maps/place//data=!4m3!3...,SGD,1,SGD,10,Asia/Singapore,SG,en-US,229495,"[500 Clemenceau Ave N, #01-20]"


In [167]:
print(test_1_2.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'dineIn',
       'servesBreakfast', 'servesLunch', 'servesDinner', 'servesBeer',
       'servesWine', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'liveMusic', 'menuForChildren', 'goodForChildren',
       'allowsDogs', 'restroom', 'goodForGroups', 'goodForWatchingSports',
       'containingPlaces', 'plusCode.globalCode', 'plusCode.compoundCode',
       'location.latitude', 'location.longitude', 'viewport.low.latitude',
       'viewport.low.longitude', 'viewport.high.latitude',
       'viewport.high.longitude', 'regularOpeningHours.openNow',
       'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions',
       'regularOpeningHours.nextCloseTime', 'displayName.text',
       'display

In [168]:
test_1_2[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units
0,Kwang Kee Teochew Fish Porridge,ChIJPbO-7-sZ2jER1zjzFT3nUyY,restaurant,"500 Clemenceau Ave N, #01-20, Singapore 229495",1.311873,103.839863,4,[{'name': 'places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/...,74,"[Monday: Closed, Tuesday: 11:00 AM – 8:00 PM, ...",1,10


In [180]:
pd.json_normalize(test_1_2['reviews'][0])   # convert reviews into a dataframe

,name,relativePublishTimeDescription,rating,publishTime,flagContentUri,googleMapsUri,text.text,text.languageCode,originalText.text,originalText.languageCode,authorAttribution.displayName,authorAttribution.uri,authorAttribution.photoUri
0,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/Ci9...,2 months ago,4,2025-08-21T09:17:16.264180882Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,Food is decent but for the accolades that they...,en,Food is decent but for the accolades that they...,en,Nicole Quek,https://www.google.com/maps/contrib/1135290671...,https://lh3.googleusercontent.com/a-/ALV-UjW2p...
1,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/Ci9...,a month ago,4,2025-09-18T12:22:25.744955888Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,"""MICHELIN, but the standard could have been hi...",en,"""MICHELIN, but the standard could have been hi...",en,"Yong Hui, Ronny Tan",https://www.google.com/maps/contrib/1121038548...,https://lh3.googleusercontent.com/a/ACg8ocIW-E...
2,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/Chd...,7 months ago,5,2025-03-27T03:12:24.362809Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,"Tried the fish soup here for the 1st time, at ...",en,"Tried the fish soup here for the 1st time, at ...",en,Joseph Loh,https://www.google.com/maps/contrib/1160500400...,https://lh3.googleusercontent.com/a-/ALV-UjUiF...
3,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/Ci9...,3 months ago,4,2025-07-27T02:31:35.557917244Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,"A very normal, clean-tasting fish soup. Nothin...",en,"A very normal, clean-tasting fish soup. Nothin...",en,K,https://www.google.com/maps/contrib/1136460660...,https://lh3.googleusercontent.com/a/ACg8ocKnc0...
4,places/ChIJPbO-7-sZ2jER1zjzFT3nUyY/reviews/ChZ...,3 years ago,3,2022-08-05T00:57:05.735843Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,Their fish soup is alright. I've had better on...,en,Their fish soup is alright. I've had better on...,en,Flo Yeow,https://www.google.com/maps/contrib/1015974848...,https://lh3.googleusercontent.com/a-/ALV-UjUCE...


In [189]:
# extract individual reviews

print(test_1_2['reviews'][0][0].get('text').get('text'))   # 1st review
print(test_1_2['reviews'][0][1].get('text').get('text'))   # 2nd review
print(test_1_2['reviews'][0][2].get('text').get('text'))   # 3rd review
print(test_1_2['reviews'][0][3].get('text').get('text'))   # 4th review
print(test_1_2['reviews'][0][4].get('text').get('text'))   # 5th review

Food is decent but for the accolades that they had displayed (Michelin listed from 2021-2025), one would have higher expectations.

I have eaten from the store in the past and I feel like their fish portions have been getting smaller. Undoubtedly, the fish was fresh and the meat was sweet. However, for $6, only five to six thin, bite-sized slices of fish were given. It’s a different case if the fish was chunky or of a thicker cut.

Soup broth is light, leans towards to sweeter side.
"MICHELIN, but the standard could have been higher.

Fish is fresh but the slices seems to have become thinner over the years.

Broth is sweet and light.

Fried Fish Soup ($6,8,10,12)
Dual Fish Soup ($7,8,10,12)"
Tried the fish soup here for the 1st time, at 1st glance thought serving was small, but after trying, wow really good, fish is definitely fresh, soup is sweet and flavourful, don't rush to add and soya sauce before you sip it, try original before adding anything except for the pepper that they adde

### 1.3. searching for hawker stalls in singapore

In [44]:
fieldmask_1_3 = '*'
textquery_1_3 = 'hawker stalls in singapore'

test_1_3 = call_text_search_api(fieldmask_1_3, textquery_1_3, api_key)

page 1: returned 20 results (total: 20)
page 2: returned 0 results (total: 20)


In [ ]:
test_1_3   # only 20 entries returned

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines,curbsidePickup,paymentOptions.acceptsCreditCards,parkingOptions.freeParkingLot,parkingOptions.freeStreetParking,accessibilityOptions.wheelchairAccessibleParking,containingPlaces
0,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84,ChIJseQsTQ0Z2jERqpBTWF0Zf84,"[tourist_attraction, restaurant, food, point_o...",6225 5632,+65 6225 5632,"1 Kadayanallur St, Singapore 069184","[{'longText': '1', 'shortText': '1', 'types': ...",4.4,https://maps.google.com/?cid=14879639582559932...,https://www.nea.gov.sg/our-services/hawker-man...,...,SG,en-US,069184,[1 Kadayanallur St],NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,"[tourist_attraction, restaurant, food, point_o...",6220 2138,+65 6220 2138,"18 Raffles Quay, Singapore 048582","[{'longText': '18', 'shortText': '18', 'types'...",4.4,https://maps.google.com/?cid=18296669017417104...,https://laupasat.sg/,...,SG,en-US,048582,[18 Raffles Quay],False,True,False,False,False,NaN
2,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,NaN,...,SG,en-US,229495,[500 Clemenceau Ave N],False,True,NaN,False,True,NaN
3,places/ChIJWzTvXEcY2jERvSzB8XNKKCk,ChIJWzTvXEcY2jERvSzB8XNKKCk,"[bakery, restaurant, food_store, food, point_o...",NaN,NaN,"51 Old Airport Rd, Singapore 390051","[{'longText': '51', 'shortText': '51', 'types'...",4.3,https://maps.google.com/?cid=29657022164611350...,https://www.nea.gov.sg/our-services/hawker-man...,...,SG,en-US,390051,[51 Old Airport Rd],NaN,False,False,False,True,NaN
4,places/ChIJ04DTdbQZ2jERFt4kBQi-E60,ChIJ04DTdbQZ2jERFt4kBQi-E60,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"505 Beach Rd, Singapore 199583","[{'longText': '505', 'shortText': '505', 'type...",4.3,https://maps.google.com/?cid=12471520734774353...,NaN,...,SG,en-US,199583,[505 Beach Rd],NaN,False,False,NaN,True,NaN
5,places/ChIJ14tfr0UW2jERvfZClMtLrsQ,ChIJ14tfr0UW2jERvfZClMtLrsQ,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...","[{'longText': 'Ci Yuan Community Centre', 'sho...",4.2,https://maps.google.com/?cid=14172348415119390...,http://ciyuanhawker.com.sg/,...,SG,en-US,538776,"[51 Hougang Ave 9, Ci Yuan Community Centre]",NaN,False,NaN,NaN,True,NaN
6,places/ChIJOU6y-HkZ2jERchOICIgjl0E,ChIJOU6y-HkZ2jERchOICIgjl0E,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"30 Seng Poh Rd, Singapore 168898","[{'longText': '30', 'shortText': '30', 'types'...",4.3,https://maps.google.com/?cid=47262854011142849...,NaN,...,SG,en-US,168898,[30 Seng Poh Rd],NaN,False,NaN,NaN,True,NaN
7,places/ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"7 Maxwell Rd, Singapore 069111","[{'longText': '7', 'shortText': '7', 'types': ...",4.3,https://maps.google.com/?cid=14179387312410727...,http://www.nea.gov.sg/,...,SG,en-US,069111,[7 Maxwell Rd],NaN,False,False,False,True,NaN
8,places/ChIJv4Mk4QYa2jERlGncaOIvpW4,ChIJv4Mk4QYa2jERlGncaOIvpW4,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"2 Adam Rd, Singapore 289876","[{'longText': '2', 'shortText': '2', 'types': ...",4.1,https://maps.google.com/?cid=79728313647977209...,https://www.facebook.com/adamfoodcentre/,...,SG,en-US,289876,[2 Adam Rd],NaN,False,False,NaN,True,NaN
9,places/ChIJsUJymcwZ2jERaYAaWRthb5s,ChIJsUJymcwZ2jERaYAaWRthb5s,"[food_court, food, point_of_interest, establis...",NaN,NaN,"2 Orchard Turn, #B4-66, Singapore 238801","[{'longText': '#B4-66', 'shortText': '#B4-66',...",4.6,https://maps.google.com/?cid=11

In [46]:
print(test_1_3.columns)

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'delivery',
       'dineIn', 'reservable', 'servesBreakfast', 'servesLunch',
       'servesDinner', 'servesBeer', 'servesWine', 'servesBrunch',
       'servesVegetarianFood', 'primaryType', 'shortFormattedAddress',
       'reviews', 'photos', 'outdoorSeating', 'liveMusic', 'menuForChildren',
       'servesDessert', 'servesCoffee', 'goodForChildren', 'allowsDogs',
       'restroom', 'goodForGroups', 'goodForWatchingSports',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpening

In [48]:
test_1_3[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Maxwell Food Centre,ChIJseQsTQ0Z2jERqpBTWF0Zf84,restaurant,"1 Kadayanallur St, Singapore 069184",1.280336,103.844767,4.4,[{'name': 'places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/...,21733,"[Monday: 8:00 AM – 10:00 PM, Tuesday: 8:00 AM ...",1,10,Chinatown hawker center with stands selling st...
1,Lau Pa Sat,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,restaurant,"18 Raffles Quay, Singapore 048582",1.280512,103.850381,4.4,[{'name': 'places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0/...,35480,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,20,Rebuilt Victorian covered hawker centre in CBD...
2,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
3,Old Airport Road Food Centre,ChIJWzTvXEcY2jERvSzB8XNKKCk,restaurant,"51 Old Airport Rd, Singapore 390051",1.308240,103.885844,4.3,[{'name': 'places/ChIJWzTvXEcY2jERvSzB8XNKKCk/...,12383,"[Monday: 6:00 AM – 10:30 PM, Tuesday: 6:00 AM ...",1,10,Busy indoor hawker center with stalls serving ...
4,Golden Mile Food Centre,ChIJ04DTdbQZ2jERFt4kBQi-E60,restaurant,"505 Beach Rd, Singapore 199583",1.303091,103.863930,4.3,[{'name': 'places/ChIJ04DTdbQZ2jERFt4kBQi-E60/...,7284,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",1,10,NaN
5,Ci Yuan Hawker Centre,ChIJ14tfr0UW2jERvfZClMtLrsQ,restaurant,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...",1.375086,103.882926,4.2,[{'name': 'places/ChIJ14tfr0UW2jERvfZClMtLrsQ/...,2462,"[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",1,10,NaN
6,Tiong Bahru Market,ChIJOU6y-HkZ2jERchOICIgjl0E,restaurant,"30 Seng Poh Rd, Singapore 168898",1.285077,103.832807,4.3,[{'name': 'places/ChIJOU6y-HkZ2jERchOICIgjl0E/...,10605,NaN,1,10,Lively food court with Singaporean hawker stal...
7,Amoy Street Food Centre,ChIJ1ZbzmxIZ2jERLxbskKJNx8Q,restaurant,"7 Maxwell Rd, Singapore 069111",1.279427,103.846796,4.3,[{'name': 'places/ChIJ1ZbzmxIZ2jERLxbskKJNx8Q/...,3471,"[Monday: 6:30 AM – 9:00 PM, Tuesday: 6:30 AM –...",1,10,Hawker center whose endless stalls offer Singa...
8,Adam Food Centre,ChIJv4Mk4QYa2jERlGncaOIvpW4,restaurant,"2 Adam Rd, Singapore 289876",1.324041,103.814155,4.1,[{'name': 'places/ChIJv4Mk4QYa2jERlGncaOIvpW4/...,7994,"[Monday: 7:00 AM – 2:00 AM, Tuesday: 7:00 AM –...",1,10,NaN
9,Hawkers' Street @ ION,ChIJsUJymcwZ2jERaYAaWRthb5s,food_court,"2 Orchard Turn, #B4-66, Singapore 238801",1.303929,103.831951,4.6,[{'name': 'places/ChIJsUJymcwZ2jERaYAaWRthb5s/...,1509,"[Monday: 10:30 AM – 9:00 PM, Tuesday: 10:30 AM...",1,10,NaN


### 1.4. searching for hawker centres in singapore

In [49]:
fieldmask_1_4 = '*'
textquery_1_4 = 'hawker centres in singapore'

test_1_4 = call_text_search_api(fieldmask_1_4, textquery_1_4, api_key)

page 1: returned 20 results (total: 20)
page 2: returned 0 results (total: 20)


In [ ]:
test_1_4   # only 20 entries returned

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,rating,googleMapsUri,websiteUri,...,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines,curbsidePickup,paymentOptions.acceptsCreditCards,parkingOptions.freeParkingLot,parkingOptions.freeStreetParking,accessibilityOptions.wheelchairAccessibleParking,containingPlaces,parkingOptions.valetParking
0,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84,ChIJseQsTQ0Z2jERqpBTWF0Zf84,"[tourist_attraction, restaurant, food, point_o...",6225 5632,+65 6225 5632,"1 Kadayanallur St, Singapore 069184","[{'longText': '1', 'shortText': '1', 'types': ...",4.4,https://maps.google.com/?cid=14879639582559932...,https://www.nea.gov.sg/our-services/hawker-man...,...,en-US,069184,[1 Kadayanallur St],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,"[tourist_attraction, restaurant, food, point_o...",6220 2138,+65 6220 2138,"18 Raffles Quay, Singapore 048582","[{'longText': '18', 'shortText': '18', 'types'...",4.4,https://maps.google.com/?cid=18296669017417104...,https://laupasat.sg/,...,en-US,048582,[18 Raffles Quay],False,True,False,False,False,NaN,NaN
2,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,NaN,...,en-US,229495,[500 Clemenceau Ave N],False,True,NaN,False,True,NaN,NaN
3,places/ChIJ04DTdbQZ2jERFt4kBQi-E60,ChIJ04DTdbQZ2jERFt4kBQi-E60,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"505 Beach Rd, Singapore 199583","[{'longText': '505', 'shortText': '505', 'type...",4.3,https://maps.google.com/?cid=12471520734774353...,NaN,...,en-US,199583,[505 Beach Rd],NaN,False,False,NaN,True,NaN,NaN
4,places/ChIJWzTvXEcY2jERvSzB8XNKKCk,ChIJWzTvXEcY2jERvSzB8XNKKCk,"[bakery, food_store, restaurant, food, point_o...",NaN,NaN,"51 Old Airport Rd, Singapore 390051","[{'longText': '51', 'shortText': '51', 'types'...",4.3,https://maps.google.com/?cid=29657022164611350...,https://www.nea.gov.sg/our-services/hawker-man...,...,en-US,390051,[51 Old Airport Rd],NaN,False,False,False,True,NaN,NaN
5,places/ChIJ14tfr0UW2jERvfZClMtLrsQ,ChIJ14tfr0UW2jERvfZClMtLrsQ,"[restaurant, food, point_of_interest, establis...",6225 5632,+65 6225 5632,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...","[{'longText': 'Ci Yuan Community Centre', 'sho...",4.2,https://maps.google.com/?cid=14172348415119390...,http://ciyuanhawker.com.sg/,...,en-US,538776,"[51 Hougang Ave 9, Ci Yuan Community Centre]",NaN,False,NaN,NaN,True,NaN,NaN
6,places/ChIJsUJymcwZ2jERaYAaWRthb5s,ChIJsUJymcwZ2jERaYAaWRthb5s,"[food_court, food, point_of_interest, establis...",NaN,NaN,"2 Orchard Turn, #B4-66, Singapore 238801","[{'longText': '#B4-66', 'shortText': '#B4-66',...",4.6,https://maps.google.com/?cid=11200277568380633...,https://ionorchard.com/shop/hawkers-street,...,en-US,238801,"[2 Orchard Turn, #B4-66]",NaN,NaN,NaN,NaN,True,[{'name': 'places/ChIJu6qq6Y0Z2jERZQZRLwnba5k'...,NaN
7,places/ChIJQ0ge3fEZ2jER3M6bSkGnWec,ChIJQ0ge3fEZ2jER3M6bSkGnWec,"[indian_restaurant, restaurant, food, point_of...",NaN,NaN,"665 Buffalo Road Zhujiao Centre, Tekka, Market...",[{'longText': '665 Buffalo Road Zhujiao Centre...,4.2,https://maps.google.com/?cid=16670539394555039...,NaN,...,en-US,210665,"[665 Buffalo Road Zhujiao Centre, Tekka, Market]",NaN,NaN,NaN,NaN,True,NaN,NaN
8,places/ChIJOU6y-HkZ2jERchOICIgjl0E,ChIJOU6y-HkZ2jERchOICIgjl0E,"[restaurant, food, point_of_interest, establis...",NaN,NaN,"30 Seng Poh Rd, Singapore 168898","[{'longText': '30', 'shortText': '30', 'types'...",4.3,https://maps.google.com/?cid=47262854011142849...,NaN,...,en-US,168898,[30 Seng Poh Rd],NaN,False,NaN,NaN,True,NaN,NaN
9,places/ChIJq-d3hRY82jERfOWbep52mCs,ChIJq-d3hRY82jERfOWbep52mCs,"[food_court, restaurant, food, point_of_intere...",NaN,NaN,"2 Changi Village Rd, Singapore 50

In [51]:
print(test_1_4.columns)

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'websiteUri', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'takeout', 'delivery',
       'dineIn', 'reservable', 'servesBreakfast', 'servesLunch',
       'servesDinner', 'servesBeer', 'servesWine', 'servesBrunch',
       'servesVegetarianFood', 'primaryType', 'shortFormattedAddress',
       'reviews', 'photos', 'outdoorSeating', 'liveMusic', 'menuForChildren',
       'servesDessert', 'servesCoffee', 'goodForChildren', 'allowsDogs',
       'restroom', 'goodForGroups', 'goodForWatchingSports',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpening

In [55]:
test_1_4[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Maxwell Food Centre,ChIJseQsTQ0Z2jERqpBTWF0Zf84,restaurant,"1 Kadayanallur St, Singapore 069184",1.280336,103.844767,4.4,[{'name': 'places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/...,21733,"[Monday: 8:00 AM – 10:00 PM, Tuesday: 8:00 AM ...",1,10,Chinatown hawker center with stands selling st...
1,Lau Pa Sat,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,restaurant,"18 Raffles Quay, Singapore 048582",1.280512,103.850381,4.4,[{'name': 'places/ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0/...,35480,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,20,Rebuilt Victorian covered hawker centre in CBD...
2,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
3,Golden Mile Food Centre,ChIJ04DTdbQZ2jERFt4kBQi-E60,restaurant,"505 Beach Rd, Singapore 199583",1.303091,103.863930,4.3,[{'name': 'places/ChIJ04DTdbQZ2jERFt4kBQi-E60/...,7284,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",1,10,NaN
4,Old Airport Road Food Centre,ChIJWzTvXEcY2jERvSzB8XNKKCk,restaurant,"51 Old Airport Rd, Singapore 390051",1.308240,103.885844,4.3,[{'name': 'places/ChIJWzTvXEcY2jERvSzB8XNKKCk/...,12383,"[Monday: 6:00 AM – 10:30 PM, Tuesday: 6:00 AM ...",1,10,Busy indoor hawker center with stalls serving ...
5,Ci Yuan Hawker Centre,ChIJ14tfr0UW2jERvfZClMtLrsQ,restaurant,"51 Hougang Ave 9, Ci Yuan Community Centre, Si...",1.375086,103.882926,4.2,[{'name': 'places/ChIJ14tfr0UW2jERvfZClMtLrsQ/...,2462,"[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",1,10,NaN
6,Hawkers' Street @ ION,ChIJsUJymcwZ2jERaYAaWRthb5s,food_court,"2 Orchard Turn, #B4-66, Singapore 238801",1.303929,103.831951,4.6,[{'name': 'places/ChIJsUJymcwZ2jERaYAaWRthb5s/...,1509,"[Monday: 10:30 AM – 9:00 PM, Tuesday: 10:30 AM...",1,10,NaN
7,Tekka Centre,ChIJQ0ge3fEZ2jER3M6bSkGnWec,restaurant,"665 Buffalo Road Zhujiao Centre, Tekka, Market...",1.306384,103.850701,4.2,[{'name': 'places/ChIJQ0ge3fEZ2jER3M6bSkGnWec/...,22515,NaN,1,10,Mixed-use market building established in 1915 ...
8,Tiong Bahru Market,ChIJOU6y-HkZ2jERchOICIgjl0E,restaurant,"30 Seng Poh Rd, Singapore 168898",1.285077,103.832807,4.3,[{'name': 'places/ChIJOU6y-HkZ2jERchOICIgjl0E/...,10605,NaN,1,10,Lively food court with Singaporean hawker stal...
9,Changi Village Hawker Centre,ChIJq-d3hRY82jERfOWbep52mCs,restaurant,"2 Changi Village Rd, Singapore 500002",1.389237,103.988301,4.3,[{'name': 'places/ChIJq-d3hRY82jERfOWbep52mCs/...,13757,"[Monday: 6:00 AM – 2:00 AM, Tuesday: 6:00 AM –...",1,10,"Vendors selling nasi lemak rice, fried bananas..."


In [181]:
pd.json_normalize(test_1_4['reviews'][0])   # convert reviews into a dataframe

,name,relativePublishTimeDescription,rating,publishTime,flagContentUri,googleMapsUri,text.text,text.languageCode,originalText.text,originalText.languageCode,authorAttribution.displayName,authorAttribution.uri,authorAttribution.photoUri
0,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/reviews/Ci9...,in the last week,4,2025-10-31T20:05:56.867074901Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,Food center with very fair prices and deliciou...,en,Food center with very fair prices and deliciou...,en,Ini,https://www.google.com/maps/contrib/1102796036...,https://lh3.googleusercontent.com/a-/ALV-UjXfV...
1,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/reviews/Ci9...,2 weeks ago,4,2025-10-17T05:49:07.125203330Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,This hawker center is conveniently located on ...,en,This hawker center is conveniently located on ...,en,M *,https://www.google.com/maps/contrib/1033308211...,https://lh3.googleusercontent.com/a/ACg8ocK3L_...
2,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/reviews/Ci9...,a week ago,4,2025-10-30T01:07:01.602730220Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,The most famous food market in Singapore. Lots...,en,The most famous food market in Singapore. Lots...,en,Jonathan van es,https://www.google.com/maps/contrib/1068377186...,https://lh3.googleusercontent.com/a-/ALV-UjW33...
3,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/reviews/Ci9...,3 weeks ago,5,2025-10-12T13:47:43.174704769Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,Maxwell Food Center is super easy to get to by...,en,Maxwell Food Center is super easy to get to by...,en,Clive G,https://www.google.com/maps/contrib/1116025335...,https://lh3.googleusercontent.com/a-/ALV-UjUNi...
4,places/ChIJseQsTQ0Z2jERqpBTWF0Zf84/reviews/Ci9...,a month ago,5,2025-09-28T07:28:33.050461391Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...,Went here 3 times during our trip. Food centre...,en,Went here 3 times during our trip. Food centre...,en,W L,https://www.google.com/maps/contrib/1052209559...,https://lh3.googleusercontent.com/a/ACg8ocKVTM...


In [ ]:
# extract individual reviews

print(test_1_4['reviews'][0][0].get('text').get('text'))   # 1st review of the 1st result of test_1_4 (maxwell food centre)
print(test_1_4['reviews'][0][1].get('text').get('text'))   # 2nd review of the 1st result of test_1_4
print(test_1_4['reviews'][0][2].get('text').get('text'))   # 3rd review of the 1st result of test_1_4
print(test_1_4['reviews'][0][3].get('text').get('text'))   # 4th review of the 1st result of test_1_4
print(test_1_4['reviews'][0][4].get('text').get('text'))   # 5th review of the 1st result of test_1_4, max number of reviews that can be returned

Food center with very fair prices and delicious food. Most stalls also accept card payments, but it’s always good to have some cash with you. We found the place to be very clean overall — even the toilets. There’s a great variety of food, and we easily found a place to sit.
This hawker center is conveniently located on top of the MRT, offering a variety of delicious food options. The famous chicken rice is highly recommended, but it’s worth noting that other chicken rice dishes are equally tasty. There are also many refreshing drinks, such as sugarcane with lemon.

One recent addition to the menu is chilli ban mian stall, which is a delightful dish featuring eggs, anchovies, fish balls, and prawn paste balls. While it may not be overly spicy, it’s well-made and worth trying. There are similar stall that also sell equally nice noodle and some dumpling.

Overall, this hawker center is a great place to explore local cuisine, with a wide range of dishes from Western to Eastern influences. 

## 2. nearby search api

In [ ]:
def call_nearby_search_api(fieldmask,
                           loc_restriction,
                           api_key,
                           included_types=[],   # set default to include all types
                           max_results=20):     # nearby search returns a max of 20 results in total
    
    url = 'https://places.googleapis.com/v1/places:searchNearby'

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    total_results = 0
    
    data = {
         'locationRestriction': loc_restriction,
         'includedTypes': included_types,
         'maxResultCount': max_results
    }
    
    response = requests.post(url, headers=headers, json=data)
            
    result = response.json()
    
    total_results = len(result.get('places', []))
    print(f"total: {total_results}")

    df = pd.json_normalize(result['places'])

    return df

### 2.1. searching around newton food centre

In [144]:
# generic search of the area around newton fc

fieldmask_2_1a = '*'
loc_restriction_2_1a = {
  "circle": {
    "center": { "latitude": 1.311989, "longitude": 103.839574},   # newton latitude & longitude from text search results
    "radius": 500
  }
}

test_2_1a = call_nearby_search_api(fieldmask_2_1a, loc_restriction_2_1a, api_key)

total: 20


In [119]:
test_2_1a

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,servesWine,servesBrunch,outdoorSeating,liveMusic,menuForChildren,servesCocktails,servesDessert,goodForWatchingSports,containingPlaces,accessibilityOptions.wheelchairAccessibleSeating
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJc25YLvQZ2jERs2ymuBdkbi8,ChIJc25YLvQZ2jERs2ymuBdkbi8,"[point_of_interest, establishment]","49 Scotts Rd, Singapore 228234","[{'longText': '49', 'shortText': '49', 'types'...",4.3,https://maps.google.com/?cid=34177792202657251...,480,"<span class=""street-address"">49 Scotts Rd</spa...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,places/ChIJQeI7N-wZ2jERnpSwVQl2b00,ChIJQeI7N-wZ2jERnpSwVQl2b00,"[hotel, wedding_venue, event_venue, lodging, p...","39 Scotts Rd, Singapore 228230","[{'longText': '39', 'shortText': '39', 'types'...",4.3,https://maps.google.com/?cid=55798082457997528...,480,"<span class=""street-address"">39 Scotts Rd</spa...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,places/ChIJ_VsMU-wZ2jERxLH9nSMgtFI,ChIJ_VsMU-wZ2jERxLH9nSMgtFI,"[government_office, point_of_interest, establi...","40 Scotts Rd, Singapore 228231","[{'longText': '40', 'shortText': '40', 'types'...",4.0,https://maps.google.com/?cid=59594235442646020...,480,"<span class=""street-address"">40 Scotts Rd</spa...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,places/ChIJpTh8I-wZ2jERT1ytyoHL3YI,ChIJpTh8I-wZ2jERT1ytyoHL3YI,"[point_of_interest, establishment]","47 Scotts Rd, Singapore 228233","[{'longText': '47', 'shortText': '47', 'types'...",4.0,https://maps.google.com/?cid=94299169530952653...,480,"<span class=""street-address"">47 Scotts Rd</spa...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,places/ChIJx18FI-wZ2jER-GBJiOvRjAw,ChIJx18FI-wZ2jER-GBJiOvRjAw,"[consultant, point_of_interest, establishment]","51 Scotts Rd, #01-01, Singapore 228241","[{'longText': '01-01', 'shortText': '01-01', '...",3.9,https://maps.google.com/?cid=90432843472865714...,480,"<span class=""street-address"">51 Scotts Rd</spa...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,places/ChIJrTuNxo8Z2jERsKVA7azFHyo,ChIJrTuNxo8Z2jERsKVA7azFHyo,"[condominium_complex, point_of_interest, estab...","6 Makeway Ave, Singapore 228606","[{'longText': '6', 'shortText': '6', 'types': ...",4.0,https://maps.google.com/?cid=30353620203764873...,480,"<span class=""street-address"">6 Makeway Ave</sp...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,places/ChIJr18Rv-4Z2jERXFDaPCqD8UU,ChIJr18Rv-4Z2jERXFDaPCqD8UU,"[subway_station, transit_station, point_of_int...",Singapore,"[{'longText': 'Singapore', 'shortText': 'SG', ...",4.0,https://maps.google.com/?cid=50399536754371011...,480,"<span class=""country-name"">Singapore</span>",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,places/ChIJ_1XUDOkZ2jER1ER-ny61Gc0,ChIJ_1XUDOkZ2jER1ER-ny61Gc0,"[chinese_restaurant, restaurant, food, point_o...","190 Keng Lee Rd, #01-02 Chui Huay Lim Club, Si...","[{'longText': '#01-02', 'shortText': '#01-02',...",4.4,https://maps.google.com/?cid=14779042864041968...,480,"<span class=""street-address"">190 Keng Lee Rd</...",OPERATIONAL,...,True,True,False,False,False,False,True,False,[{'name': 'places/ChIJ1zpDHs0Z2jERbu-6xMSZuoA'...,True
9,places/ChIJ1zpDHs0Z2jERbu-6xMSZuoA,ChIJ1zpDHs0Z2jERbu-6xMSZuoA,"[point_of_interest, establishment]","190 Keng Lee Rd, #04-01, Singapore 308409","[{'longText': '04-01', 'shortText': '04-01', '...",4.3,https://maps.google.com/?cid=92758954529161828...,480,"<span class=""street-address"">190 Keng Lee Rd</...",OPERATIONAL,...,NaN

In [120]:
print(test_2_1a.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [121]:
test_2_1a[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521.0,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
1,Newton MRT Station (NS21),ChIJc25YLvQZ2jERs2ymuBdkbi8,NaN,"49 Scotts Rd, Singapore 228234",1.312509,103.837952,4.3,[{'name': 'places/ChIJc25YLvQZ2jERs2ymuBdkbi8/...,451.0,NaN,NaN,NaN,NaN
2,Sheraton Towers Singapore,ChIJQeI7N-wZ2jERnpSwVQl2b00,hotel,"39 Scotts Rd, Singapore 228230",1.311824,103.836476,4.3,[{'name': 'places/ChIJQeI7N-wZ2jERnpSwVQl2b00/...,3190.0,NaN,NaN,NaN,"Plush rooms & suites, some with butler service..."
3,Environment Building,ChIJ_VsMU-wZ2jERxLH9nSMgtFI,government_office,"40 Scotts Rd, Singapore 228231",1.310640,103.836304,4.0,[{'name': 'places/ChIJ_VsMU-wZ2jERxLH9nSMgtFI/...,67.0,"[Monday: 8:30 AM – 6:00 PM, Tuesday: 8:30 AM –...",NaN,NaN,NaN
4,Goldbell Towers,ChIJpTh8I-wZ2jERT1ytyoHL3YI,NaN,"47 Scotts Rd, Singapore 228233",1.312301,103.837036,4.0,[{'name': 'places/ChIJpTh8I-wZ2jERT1ytyoHL3YI/...,247.0,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",NaN,NaN,NaN
5,Prudential @ Scotts,ChIJx18FI-wZ2jER-GBJiOvRjAw,NaN,"51 Scotts Rd, #01-01, Singapore 228241",1.312733,103.837307,3.9,[{'name': 'places/ChIJx18FI-wZ2jER-GBJiOvRjAw/...,156.0,"[Monday: 8:00 AM – 5:30 PM, Tuesday: 8:00 AM –...",NaN,NaN,NaN
6,Kopar at Newton,ChIJrTuNxo8Z2jERsKVA7azFHyo,condominium_complex,"6 Makeway Ave, Singapore 228606",1.312257,103.841626,4.0,[{'name': 'places/ChIJrTuNxo8Z2jERsKVA7azFHyo/...,39.0,NaN,NaN,NaN,NaN
7,Newton,ChIJr18Rv-4Z2jERXFDaPCqD8UU,subway_station,Singapore,1.312320,103.837990,4.0,[{'name': 'places/ChIJr18Rv-4Z2jERXFDaPCqD8UU/...,115.0,NaN,NaN,NaN,NaN
8,Chui Huay Lim Teochew Cuisine 醉花林品潮轩,ChIJ_1XUDOkZ2jER1ER-ny61Gc0,chinese_restaurant,"190 Keng Lee Rd, #01-02 Chui Huay Lim Club, Si...",1.314079,103.841530,4.4,[{'name': 'places/ChIJ_1XUDOkZ2jER1ER-ny61Gc0/...,2672.0,"[Monday: 11:30 AM – 2:30 PM, 5:30 – 11:00 PM, ...",100,NaN,NaN
9,Chui Huay Lim Club,ChIJ1zpDHs0Z2jERbu-6xMSZuoA,NaN,"190 Keng Lee Rd, #04-01, Singapore 308409",1.313904,103.841465,4.3,[{'name': 'places/ChIJ1zpDHs0Z2jERbu-6xMSZuoA/...,247.0,"[Monday: 11:30 AM – 2:30 PM, 5:30 – 11:30 PM, ...",NaN,NaN,NaN


In [139]:
# trying to get hawker stalls in newton fc using nearby search

fieldmask_2_1b = '*'
loc_restriction_2_1b = {
  "circle": {
    "center": { "latitude": 1.311989, "longitude": 103.839574},
    "radius": 50   # smaller radius to limit search within newton fc
  }
}
included_types_2_1b = ['restaurant', 'food_court']   # 'hawker_stall' is not a supported type, need to use generic types (e.g. 'restaurant', 'cafe')

test_2_1b = call_nearby_search_api(fieldmask_2_1b, loc_restriction_2_1b, api_key, included_types_2_1b)

total: 20


In [140]:
test_2_1b

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,allowsDogs,regularOpeningHours.nextOpenTime,currentOpeningHours.nextOpenTime,accessibilityOptions.wheelchairAccessibleSeating,servesBrunch,websiteUri,servesVegetarianFood,servesBreakfast,currentOpeningHours.specialDays,servesCoffee
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJx3TNyV8Z2jERAFrgrjj3HVA,ChIJx3TNyV8Z2jERAFrgrjj3HVA,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-31 Food Centre, Sing...","[{'longText': '#01-31', 'shortText': '#01-31',...",4.4,https://maps.google.com/?cid=57730421201830118...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,places/ChIJu3wK9-sZ2jERz3jrVfP4sWM,ChIJu3wK9-sZ2jERz3jrVfP4sWM,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-27 Newton Food Centr...","[{'longText': '#01-27', 'shortText': '#01-27',...",3.6,https://maps.google.com/?cid=71837966046348883...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,places/ChIJrzNA5esZ2jERJ-SPNsTOfxo,ChIJrzNA5esZ2jERJ-SPNsTOfxo,"[restaurant, food, point_of_interest, establis...","22, Newton Food Centre, Singapore 229495","[{'longText': '22', 'languageCode': 'en'}, {'l...",4.3,https://maps.google.com/?cid=19094721091526912...,480,"22, Newton Food Centre, <span class=""country-n...",OPERATIONAL,...,False,2025-11-06T07:00:00Z,2025-11-06T07:00:00Z,True,NaN,NaN,NaN,NaN,NaN,NaN
4,places/ChIJxcbG7-sZ2jERiyhLwTAA5i8,ChIJxcbG7-sZ2jERiyhLwTAA5i8,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-19 Newton Food Centr...","[{'longText': '01-19', 'shortText': '01-19', '...",4.7,https://maps.google.com/?cid=34514463738274223...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,2025-11-06T08:30:00Z,2025-11-06T08:30:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,places/ChIJU_PRvacZ2jERC2QJ764xZRk,ChIJU_PRvacZ2jERC2QJ764xZRk,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, Newton Circus, #01-73 Fo...","[{'longText': '500 Clemenceau Ave N', 'languag...",4.0,https://maps.google.com/?cid=18299234510004152...,480,"500 Clemenceau Ave N, <span class=""street-addr...",OPERATIONAL,...,NaN,2025-11-06T09:30:00Z,2025-11-06T09:30:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,places/ChIJmzPtVnQe2jEROaYw5BZn7sQ,ChIJmzPtVnQe2jEROaYw5BZn7sQ,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-28 Newton Food Centr...","[{'longText': '01-28', 'shortText': '01-28', '...",3.9,https://maps.google.com/?cid=14190392823906346...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,2025-11-06T10:00:00Z,2025-11-06T10:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,places/ChIJJczvxGIZ2jER4wtULz2lCbg,ChIJJczvxGIZ2jER4wtULz2lCbg,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-13 Newton Food Centr...","[{'longText': '01-13', 'shortText': '01-13', '...",3.7,https://maps.google.com/?cid=13261312259974761...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
8,places/ChIJRR4pnVEZ2jERdydSSSfVeYY,ChIJRR4pnVEZ2jERdydSSSfVeYY,"[restaurant, food, point_of_interest, establis...","500 Clemenceau Ave N, #01-76 Newton Food Centr...","[{'longText': '01-76', 'shortText': '01-76', '...",3.6,https://maps.google.com/?cid=96900104379748944...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,False,2025-11-06T10:00:00Z,2025-11-06T10:00:00Z,True,NaN,https://

In [141]:
print(test_2_1b.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpeningHours.weekdayDescriptions', 'e

In [ ]:
test_2_1b[['displayName.text',
          'id',
          'primaryType',   # hawker stalls mostly labelled as restaurant
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...
1,31 Heng Heng BBQ 兴兴烧烤,ChIJx3TNyV8Z2jERAFrgrjj3HVA,restaurant,"500 Clemenceau Ave N, #01-31 Food Centre, Sing...",1.311950,103.839577,4.4,[{'name': 'places/ChIJx3TNyV8Z2jERAFrgrjj3HVA/...,111,"[Monday: 12:00 – 10:30 PM, Tuesday: Closed, We...",NaN,NaN,NaN
2,Alliance Seafood,ChIJu3wK9-sZ2jERz3jrVfP4sWM,restaurant,"500 Clemenceau Ave N, #01-27 Newton Food Centr...",1.311986,103.839729,3.6,[{'name': 'places/ChIJu3wK9-sZ2jERz3jrVfP4sWM/...,188,"[Monday: Closed, Tuesday: 11:30 AM – 10:30 PM,...",NaN,NaN,NaN
3,Best Satay,ChIJrzNA5esZ2jERJ-SPNsTOfxo,restaurant,"22, Newton Food Centre, Singapore 229495",1.311995,103.839828,4.3,[{'name': 'places/ChIJrzNA5esZ2jERJ-SPNsTOfxo/...,63,"[Monday: Closed, Tuesday: 3:00 – 11:30 PM, Wed...",1,10,NaN
4,Stingray Goh Chew Kee Seafood,ChIJxcbG7-sZ2jERiyhLwTAA5i8,restaurant,"500 Clemenceau Ave N, #01-19 Newton Food Centr...",1.311868,103.839835,4.7,[{'name': 'places/ChIJxcbG7-sZ2jERiyhLwTAA5i8/...,41,"[Monday: 4:30 – 11:30 PM, Tuesday: 4:30 – 11:3...",10,20,NaN
5,Hup Kee Fried Oyster Omelette,ChIJU_PRvacZ2jERC2QJ764xZRk,restaurant,"500 Clemenceau Ave N, Newton Circus, #01-73 Fo...",1.312241,103.839583,4.0,[{'name': 'places/ChIJU_PRvacZ2jERC2QJ764xZRk/...,111,"[Monday: Closed, Tuesday: Closed, Wednesday: 5...",1,10,NaN
6,Heng Carrot Cake & Oyster Omelette,ChIJmzPtVnQe2jEROaYw5BZn7sQ,restaurant,"500 Clemenceau Ave N, #01-28 Newton Food Centr...",1.311979,103.839680,3.9,[{'name': 'places/ChIJmzPtVnQe2jEROaYw5BZn7sQ/...,125,"[Monday: 6:00 – 11:00 PM, Tuesday: Closed, Wed...",1,10,NaN
7,Kwee Heng,ChIJJczvxGIZ2jER4wtULz2lCbg,restaurant,"500 Clemenceau Ave N, #01-13 Newton Food Centr...",1.311854,103.839706,3.7,[{'name': 'places/ChIJJczvxGIZ2jER4wtULz2lCbg/...,93,"[Monday: 9:00 AM – 9:00 PM, Tuesday: 9:00 AM –...",1,10,NaN
8,R&B Express,ChIJRR4pnVEZ2jERdydSSSfVeYY,restaurant,"500 Clemenceau Ave N, #01-76 Newton Food Centr...",1.312165,103.839664,3.6,[{'name': 'places/ChIJRR4pnVEZ2jERdydSSSfVeYY/...,73,"[Monday: 6:00 PM – 3:00 AM, Tuesday: 6:00 PM –...",1,10,NaN
9,thatsmypizza.sg @Newton,ChIJAz99K7UZ2jERy-Mbx9lekuY,meal_takeaway,"500 Clemenceau Ave N, #01-67 Newton Food Centr...",1.312326,103.839481,4.8,[{'name': 'places/ChIJAz99K7UZ2jERy-Mbx9lekuY/...,67,"[Monday: Closed, Tuesday: 6:00 – 10:30 PM, Wed...",20,30,NaN


## 3. place details api

In [145]:
def call_place_details_api(fieldmask, place_id, api_key):

    url = 'https://places.googleapis.com/v1/places/' + place_id

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': fieldmask
    }

    response = requests.get(url, headers=headers)   # get retrieves a single resource based on a unique id
    result = response.json()

    df = pd.json_normalize(result)

    return df

### 3.1. newton food centre

In [149]:
fieldmask_3_1 = '*'
id_3_1 = 'ChIJe4wL_OsZ2jERySoDk6jlhNQ'   # newton food centre place id from text / nearby search result

test_3_1 = call_place_details_api(fieldmask_3_1, id_3_1, api_key)

In [150]:
test_3_1

,name,id,types,formattedAddress,addressComponents,rating,googleMapsUri,utcOffsetMinutes,adrFormatAddress,businessStatus,...,googleMapsLinks.photosUri,priceRange.startPrice.currencyCode,priceRange.startPrice.units,priceRange.endPrice.currencyCode,priceRange.endPrice.units,timeZone.id,postalAddress.regionCode,postalAddress.languageCode,postalAddress.postalCode,postalAddress.addressLines
0,places/ChIJe4wL_OsZ2jERySoDk6jlhNQ,ChIJe4wL_OsZ2jERySoDk6jlhNQ,"[food_court, restaurant, food, point_of_intere...","500 Clemenceau Ave N, Singapore 229495","[{'longText': '500', 'shortText': '500', 'type...",4.2,https://maps.google.com/?cid=15313617145150253...,480,"<span class=""street-address"">500 Clemenceau Av...",OPERATIONAL,...,https://www.google.com/maps/place//data=!4m3!3...,SGD,10,SGD,30,Asia/Singapore,SG,en-US,229495,[500 Clemenceau Ave N]


In [151]:
print(test_3_1.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn',
       'curbsidePickup', 'primaryType', 'shortFormattedAddress', 'reviews',
       'photos', 'goodForChildren', 'restroom', 'goodForGroups',
       'pureServiceAreaBusiness', 'plusCode.globalCode',
       'plusCode.compoundCode', 'location.latitude', 'location.longitude',
       'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions', 'displayName.text',
       'displayName.languageCode', 'primaryTypeDisplayName.text',
       'primaryTypeDisplayName.languageCode', 'currentOpeningHours.openNow',
       'currentOpeningHours.periods',
       'currentOpe

In [178]:
test_3_1[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units',
          'editorialSummary.text',
          'addressDescriptor.landmarks',
          'addressDescriptor.areas']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units,editorialSummary.text,addressDescriptor.landmarks,addressDescriptor.areas
0,Newton Food Centre,ChIJe4wL_OsZ2jERySoDk6jlhNQ,food_court,"500 Clemenceau Ave N, Singapore 229495",1.311989,103.839574,4.2,[{'name': 'places/ChIJe4wL_OsZ2jERySoDk6jlhNQ/...,14521,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",10,30,Lively market with numerous stalls & hawkers o...,[{'name': 'places/ChIJ6wSMeewZ2jERnRkQnVf_7-c'...,[{'name': 'places/ChIJHdG18usZ2jERB9tMG6Pfyg0'...


In [164]:
pd.json_normalize(test_3_1['addressDescriptor.landmarks'][0])   # convert address descriptor landmarks into a dataframe

,name,placeId,types,straightLineDistanceMeters,travelDistanceMeters,displayName.text,displayName.languageCode
0,places/ChIJ6wSMeewZ2jERnRkQnVf_7-c,ChIJ6wSMeewZ2jERnRkQnVf_7-c,"[establishment, point_of_interest]",147.41432,199.01514,Cairnhill Community Club,en
1,places/ChIJSWG3WbkZ2jERZY4iU6PnhUQ,ChIJSWG3WbkZ2jERZY4iU6PnhUQ,"[establishment, point_of_interest]",195.15062,176.18076,Lasalle College Of The Arts (Winstedt Campus),en
2,places/ChIJCyLqIusZ2jERKLxdCy-uU78,ChIJCyLqIusZ2jERKLxdCy-uU78,"[establishment, point_of_interest, primary_sch...",362.05005,334.09598,Anglo-Chinese School (Junior),en
3,places/ChIJc25YLvQZ2jERs2ymuBdkbi8,ChIJc25YLvQZ2jERs2ymuBdkbi8,"[establishment, point_of_interest]",212.39235,306.30728,Newton MRT Station (NS21),en
4,places/ChIJ1zpDHs0Z2jERbu-6xMSZuoA,ChIJ1zpDHs0Z2jERbu-6xMSZuoA,"[establishment, point_of_interest]",327.50693,385.62897,Chui Huay Lim Club,en


In [179]:
pd.json_normalize(test_3_1['addressDescriptor.areas'][0])   # convert address descriptor areas into a dataframe

,name,placeId,containment,displayName.text,displayName.languageCode
0,places/ChIJHdG18usZ2jERB9tMG6Pfyg0,ChIJHdG18usZ2jERB9tMG6Pfyg0,OUTSKIRTS,Parking lot,en
1,places/ChIJ4c17uewZ2jERgKbarqz3AAU,ChIJ4c17uewZ2jERgKbarqz3AAU,OUTSKIRTS,Newton,en
2,places/ChIJ98TwBAgZ2jERDemIweIVO70,ChIJ98TwBAgZ2jERDemIweIVO70,OUTSKIRTS,Central Area,en


### 3.2. specific stall in newton fc

In [197]:
fieldmask_3_2a = '*'
id_3_2a = 'ChIJrzNA5esZ2jERJ-SPNsTOfxo'   # best satay (in newton fc) place id from text / nearby search result

test_3_2a = call_place_details_api(fieldmask_3_2a, id_3_2a, api_key)

In [198]:
print(test_3_2a.columns)

Index(['name', 'id', 'types', 'formattedAddress', 'addressComponents',
       'rating', 'googleMapsUri', 'utcOffsetMinutes', 'adrFormatAddress',
       'businessStatus', 'userRatingCount', 'iconMaskBaseUri',
       'iconBackgroundColor', 'takeout', 'delivery', 'dineIn', 'servesDinner',
       'servesBeer', 'servesWine', 'primaryType', 'shortFormattedAddress',
       'reviews', 'photos', 'liveMusic', 'menuForChildren', 'servesDessert',
       'goodForChildren', 'allowsDogs', 'restroom', 'goodForGroups',
       'goodForWatchingSports', 'containingPlaces', 'pureServiceAreaBusiness',
       'plusCode.globalCode', 'plusCode.compoundCode', 'location.latitude',
       'location.longitude', 'viewport.low.latitude', 'viewport.low.longitude',
       'viewport.high.latitude', 'viewport.high.longitude',
       'regularOpeningHours.openNow', 'regularOpeningHours.periods',
       'regularOpeningHours.weekdayDescriptions',
       'regularOpeningHours.nextCloseTime', 'displayName.text',
       'displa

In [199]:
test_3_2a[['displayName.text',
          'id',
          'primaryType',
          'formattedAddress',
          'location.latitude',
          'location.longitude',
          'rating',
          'reviews',
          'userRatingCount',
          'regularOpeningHours.weekdayDescriptions',
          'priceRange.startPrice.units',
          'priceRange.endPrice.units']]

,displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units
0,Best Satay,ChIJrzNA5esZ2jERJ-SPNsTOfxo,restaurant,"22, Newton Food Centre, Singapore 229495",1.311995,103.839828,4.3,[{'name': 'places/ChIJrzNA5esZ2jERJ-SPNsTOfxo/...,63,"[Monday: Closed, Tuesday: 3:00 – 11:30 PM, Wed...",1,10


In [200]:
# specify desired fields instead of returning all available fields
fieldmask_3_2b = 'displayName.text,id,primaryType,formattedAddress,location.latitude,location.longitude,rating,reviews,userRatingCount,regularOpeningHours.weekdayDescriptions,priceRange.startPrice.units,priceRange.endPrice.units'
id_3_2b = 'ChIJrzNA5esZ2jERJ-SPNsTOfxo'

test_3_2b = call_place_details_api(fieldmask_3_2b, id_3_2b, api_key)

In [201]:
test_3_2b

,id,formattedAddress,rating,userRatingCount,primaryType,reviews,location.latitude,location.longitude,regularOpeningHours.weekdayDescriptions,displayName.text,priceRange.startPrice.units,priceRange.endPrice.units
0,ChIJrzNA5esZ2jERJ-SPNsTOfxo,"22, Newton Food Centre, Singapore 229495",4.3,63,restaurant,[{'name': 'places/ChIJrzNA5esZ2jERJ-SPNsTOfxo/...,1.311995,103.839828,"[Monday: Closed, Tuesday: 3:00 – 11:30 PM, Wed...",Best Satay,1,10
